Первым делом скачаем обе таблицы

In [136]:
import sys
from natasha import DatesExtractor
import re
from numpy import prod

In [8]:
!"{sys.executable}" -m pip install google-cloud-bigquery

In [9]:
from google.cloud import bigquery

В клиент вставьте свой аккаунт Google Cloud Platform.

In [10]:
client = bigquery.Client.from_service_account_json("D:\Data-Science-lecture-a42c6d519bb9.json")

Первым делом мы сформируем запрос на русском языке. В этом запросе укажите время создания интересующего вас обекта(пожалуйста не пишите век, natasha их не распознаёт). Вернётся весь массив с точной датой (он выбирает основываясь на границах времени, но выдаёт более точный отрезок) и некоторую другую информацию. Я использовал 1956 год. В нём не так много данных

In [45]:
input_information = input()
extractor = DatesExtractor()
Date_request = extractor(input_information)
Date_to_found = Date_request[0].fact.as_json["year"]

19 октября 1956 года


In [66]:
df_of_objects  = client.query("""
SELECT object_name, title, culture, object_date, country, link_resource FROM  `bigquery-public-data.the_met.objects` 
WHERE object_begin_date <= {DATE} AND object_end_date >= {DATE} AND object_date IS NOT NULL
""".format(DATE=Date_to_found, )).to_dataframe() # из лекции можно вспомнить, что такой метод не очень надёжный, но так как Date_to_found точно будет числом, использовать его вполне безопасно.
df_of_objects = df_of_objects.style.set_properties( **{'width': '200px'})

In [67]:
df_of_objects

,object_name,title,culture,object_date,country,link_resource
0,Fan,None,Japan,20th century,None,http://www.metmuseum.org/art/collection/search...
1,Fan,None,Japan,20th century,None,http://www.metmuseum.org/art/collection/search...
2,Fan,None,Japan,20th century,None,http://www.metmuseum.org/art/collection/search...
3,Fan,None,Japan,20th century,None,http://www.metmuseum.org/art/collection/search...
4,Fan,None,Japan,20th century,None,http://www.metmuseum.org/art/collection/search...
5,Fan,None,Japan,20th century,None,http://www.metmuseum.org/art/collection/search...
6,Fan,None,Japan,20th century,None,http://www.metmuseum.org/art/collection/search...
7,Fan,None,Japan,20th century,None,http://www.metmuseum.org/art/collection/search...
8,Fan,None,Japan,20th century,None,http://www.metmuseum.org/art/collection/search...
9,Fan,None,Japan,20th century,None,http://www.metmuseum.org/art/collection/search...


Теперь давайте найдём дополнително все фотографии этих объектов хранящиеся в базе данных.
Некоторые ссылки выдают оштбку 404. Это из-за того, что база данных немного устарела.

In [80]:
df_of_images  = client.query("""
SELECT object_name, BQTMobj.title, culture, object_date, country, link_resource, original_image_url FROM  `bigquery-public-data.the_met.objects` as BQTMobj
LEFT JOIN `bigquery-public-data.the_met.images` as BQTMimg ON BQTMobj.object_id = BQTMimg.object_id
WHERE object_begin_date <= {DATE} AND object_end_date >= {DATE} AND object_date IS NOT NULL
ORDER BY BQTMobj.object_id
""".format(DATE=Date_to_found, )).to_dataframe()
df_of_images = df_of_images.style.set_properties( **{'width': '200px'})

Теперь мне хотелось бы найти самое позднее, самое ранные и среднее время надодок для каждой из культур.

In [ ]:
client.query("""
SELECT culture, MIN(object_begin_date) AS Earliest, AVG((object_begin_date+object_end_date)/2) AS Average, MAX(object_end_date) AS Latest FROM `bigquery-public-data.the_met.objects`
GROUP BY culture
""").to_dataframe()

Найдём культуры экспонаты для которой принадлежат только нашей эре и отдельно только до н.э.

In [83]:
client.query("""
SELECT culture, MIN(object_begin_date) AS Earliest, AVG((object_begin_date+object_end_date)/2) AS Average, MAX(object_end_date) AS Latest FROM `bigquery-public-data.the_met.objects`
GROUP BY culture
HAVING MIN(object_begin_date)>0
""").to_dataframe()


,culture,f0_,f1_,f2_
0,"New York: [s.n.], [18--]",1800,1805.000000,1810
1,Tibet,1,1508.626190,1999
2,"Afghanistan, possibly of West Indian manufacture",1,150.000000,299
3,Tibet (or Nepal),1450,1550.000000,1650
4,Nepal,1004,1632.740741,1899
5,"India (Jammu and Kashmir, ancient kingdom of K...",600,699.500000,799
6,Sri Lanka,400,1169.750000,1799
7,"India (Jammu and Kashmir, ancient kingdom of K...",9,690.722222,1050
8,India (Orissa?),1650,1750.000000,1850
9,India (Orissa),1100,1570.708333,1840


In [84]:
client.query("""
SELECT culture, MIN(object_begin_date) AS Earliest, AVG((object_begin_date+object_end_date)/2) AS Average, MAX(object_end_date) AS Latest FROM `bigquery-public-data.the_met.objects`
GROUP BY culture
HAVING MAX(object_end_date)<0
""").to_dataframe()


,culture,f0_,f1_,f2_
0,Northwest China,-1300,-345.102273,-1
1,"Volga region, Russia",-799,-699.500000,-600
2,Early Bronze Age,-1750,-1625.000000,-1500
3,Halstatt,-900,-758.333333,-500
4,Bronze Age,-1200,-1025.000000,-800
5,Late Halstatt Period,-599,-549.500000,-500
6,Halstatt Period,-699,-649.500000,-600
7,Late Bronze Age,-1200,-1000.000000,-800
8,European Bronze Age,-1300,-979.041667,-700
9,Celto-Iberian,-499,-449.500000,-400


Теперь воспользуемся регулярными выражениями и попробуем найти все экспонаты у которых отображены все 3 размера в сантиметрах, а затем нйдём самые большой и маленький из этих объектов.

In [115]:
df = client.query("""
SELECT dimensions FROM `bigquery-public-data.the_met.objects`
WHERE dimensions IS NOT NULL
""").to_dataframe()

In [131]:
list_of_dims = df["dimensions"]
list_of_3dim = []
for dim in list_of_dims:
    dim =  re.findall("(?:\d+(?:\.\d*)?|\.\d+) × (?:\d+(?:\.\d*)?|\.\d+) × (?:\d+(?:\.\d*)?|\.\d+)", dim)
    if len(dim)>0 :
        list_of_3dim.extend(dim)

In [145]:
list_of_volumes = []
for dim3 in list_of_3dim:
    d = dim3.split(' × ')
    d = [float(i) for i in d]
    list_of_volumes.append(prod(d))
print(" max - ",  max(list_of_volumes), "cm^3\n","min - ",  
min(list_of_volumes), "cm^3")

 max -  193186515.83999997 cm^3
 min -  0.06 cm^3
